# Imports


In [3]:
import os

from langchain_ollama import ChatOllama

os.environ["NO_PROXY"] = "localhost,127.0.0.1"
os.environ["USER_AGENT"] = "myagent"

# LLM Initialisierung mit Beispiel


In [4]:
llm = ChatOllama(
    model="gpt-oss:20b-cloud",
    base_url="http://localhost:11434",
    temperature=0.7,
)

messages = [
    ("system", "Du bist ein hilfreicher Assistent."),
    ("human", "Erkläre mir die Vorteile eines lokalen LLM."),
]

response = llm.invoke(messages)
print(response.content)

### Vorteile eines lokalen Large Language Models (LLM)

| # | Vorteil | Warum das wichtig ist | Praktische Beispiele |
|---|---------|----------------------|----------------------|
| 1 | **Datenschutz & Vertraulichkeit** | Alle Daten bleiben im Unternehmensnetzwerk – keine sensiblen Infos gehen an Dritte. | Ein Finanzinstitut kann Kundendaten für Kreditentscheidungen analysieren, ohne die Daten an einen Cloud‑Provider zu senden. |
| 2 | **Rechtliche & regulatorische Compliance** | Einhaltung von DSGVO, HIPAA, ISO‑Normen etc. ist einfacher, wenn keine Daten das Land verlassen. | Ein Gesundheitsdienstleister kann Patientenberichte verarbeiten, ohne sich um Export‑Beschränkungen sorgen zu müssen. |
| 3 | **Niedrigere Latenz & höhere Performance** | Lokale Modelle liefern Antworten in Millisekunden – ideal für Echtzeit‑Anwendungen (Chat‑Bots, Assistenz‑Apps). | Ein Call‑Center‑System beantwortet Kundenanfragen sofort, ohne Netzwerk‑Round‑Trips. |
| 4 | **Kostenkontrolle** | Nach dem einmal

# Embedding Modell Initialisierung


# Vektordatenbank Initialisierung

### Eine Collection für eine Website und eine für ein PDF-Dokument erstellen. Anschließend Collections anzeigen lassen.


### Optional: Vorherige Sammlung löschen


In [ ]:
client.delete_collection(collection_name=name_der_sammlung)

# Website laden und Text extrahieren (Website)


### Text in Chunks aufteilen (Website)


### Chunks in Vektordatenbank speichern (Website)


# PDF laden und Text extrahieren (PDF)


### Text in Chunks aufteilen (PDF)


### Chunks in Vektordatenbank speichern (PDF)


# LLM mit Retrieval Augmented Generation verwenden


### Website


### PDF
